<a href="https://colab.research.google.com/github/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting/blob/main/notebooks/n_beats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

from google.colab import userdata
token = userdata.get('GITHUB_TOKEN')
user_name = userdata.get('GITHUB_USERNAME')
mail = userdata.get('GITHUB_MAIL')

!git config --global user.name "{user_name}"
!git config --global user.email "{mail}"
!git clone https://{token}@github.com/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting.git

%cd Walmart-Recruiting-Store-Sales-Forecasting

from google.colab import userdata
! pip install -r ./requirements.txt
kaggle_json_path = userdata.get('KAGGLE_JSON_PATH')
! ./src/data_loader.sh -f {kaggle_json_path}

In [4]:
# **Torch**

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
import torch
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS
from neuralforecast.losses.pytorch import MSE

from src.config import *

stores = pd.read_csv(STORES_PATH)
features = pd.read_csv(FEATURES_PATH)
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

train['Date'] = pd.to_datetime(train.Date)
# test['Date'] = pd.to_datetime(test.Date)

In [5]:
from src.processing import split_data
x_train,y_train,x_val,y_val = split_data(train)

**Cross Validation**

In [15]:
class CustomNBEATS(NBEATS):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.optimizer = torch.optim.AdamW(self.parameters(), lr=1e-3)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.9)

    def set_optim(self,optimizer):
      self.optimizer = optimizer

    def set_scheduler(self,scheduler):
      self.scheduler = scheduler

    def configure_optimizers(self):
        # Define your custom optimizer here
        optimizer = self.optimizer #torch.optim.AdamW(self.parameters(), lr=1e-3)

        # Optional: add scheduler if needed
        scheduler = self.scheduler #torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'interval': 'epoch',
                'frequency': 1
            }
        }

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin

class Preprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        x_copy = X.copy()
        if 'y' not in x_copy.columns:
            x_copy['y'] = 0
        x_copy['Date'] = pd.to_datetime(x_copy['Date'])
        return x_copy


In [11]:
class NBEATSModel(BaseEstimator, TransformerMixin):
    def __init__(self,models:list[NBEATS]=None):
        if models == None or len(models)==0:
          if torch.cuda.is_available():
            device = 'gpu'
          else:
            device = 'cpu'
          models = [CustomNBEATS(
            input_size=15,
            h=48,
            max_steps=25 * 104,
            batch_size=64,
            stack_types=(['identity', 'trend', 'seasonality']),
            n_blocks = [1,1,1],
            random_seed=42,
            accelerator=device,
            devices=1,
            logger=True,
            enable_progress_bar=False,
            enable_model_summary=False,
        )]
        self.nf = NeuralForecast(models=models, freq='W-FRI')

    def fit(self, X, y=None):
        df = self.create_data_nbeats(X,y)
        df.sort_values(by=['ds'],inplace=True)
        self.nf.fit(df)

        return self

    def transform(self, X):
        preds = self.nf.predict()

        real = self.create_data_nbeats(X,X.y)
        merged = pd.merge(real, preds, on=['unique_id','ds'],how='left')
        # print(merged.shape)
        merged.fillna(0,inplace=True)


        return merged


    def predict(self,X=None):
        return self.transform(X)

    def create_data_nbeats(self,x:pd.DataFrame,y:pd.DataFrame) -> pd.DataFrame:
        df = pd.DataFrame()

        df = df.reset_index()
        df["Date"] = x.Date
        df["Store"] = x.Store
        df["Dept"] = x.Dept
        df["IsHoliday"] = x.IsHoliday.astype(int)
        df.rename(columns={"Date": "ds"}, inplace=True)
        df["unique_id"] = df["Store"].astype(str) + "_" + df["Dept"].astype(str)
        df["y"] = y.values
        df = df[["unique_id", "ds", "y",'IsHoliday']].copy()
        return df

In [12]:
from os import pipe
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('preprocess', Preprocessor()),
    ('model', NBEATSModel())
])

model = pipeline.fit(x_train,y_train)

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2600` reached.


In [14]:
from src.utils import wmae
x_val['y'] = y_val
preds = model.predict(x_val)
wmae(preds['y'],preds['CustomNBEATS'],preds['IsHoliday'])

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


np.float64(2373.6844628731565)

# **Grid Search**

In [16]:
l = [0.9,0.85,0.8,0.75,0.7]
for lr in l:
  estim = CustomNBEATS(
              input_size=15,
              h=48,
              max_steps=25 * 104,
              batch_size=64,
              stack_types=(['identity', 'trend', 'seasonality']),
              n_blocks = [1,1,1],
              random_seed=42,
              accelerator='gpu',
              devices=1,
              logger=True,
              enable_progress_bar=False,
              enable_model_summary=False,)

  # estim.set_optim(torch.optim.AdamW(estim.parameters(), lr=1e-3,weight_decay=lr))
  estim.set_scheduler(torch.optim.lr_scheduler.StepLR(estim.optimizer, step_size=10, gamma=lr))

  estimators = [estim]

  pipeline = Pipeline([
      ('preprocess', Preprocessor()),
      ('model', NBEATSModel(estimators))
  ])

  model = pipeline.fit(x_train,y_train)
  x_val['y'] = y_val
  preds = model.predict(x_val)
  score = wmae(preds['y'],preds['CustomNBEATS'],preds['IsHoliday'])
  print(lr,score)

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2600` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transf

0.9 2373.6844628731565


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2600` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 i

0.85 2384.477016112778


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2600` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 i

...

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2600` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 i

0.75 2344.625498683203


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2600` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


0.7 2351.841830138327


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [36]:

if torch.cuda.is_available():
  device = 'gpu'
else:
  device = 'cpu'


estim = CustomNBEATS(
            input_size=15,
            h=48,
            max_steps=100 * 104,
            batch_size=64,
            stack_types=(['identity', 'trend', 'seasonality']),
            n_blocks = [1,1,1],
            random_seed=42,
            accelerator=device,
            devices=1,
            logger=True,
            enable_progress_bar=False,
            enable_model_summary=False,)

estim.set_optim(torch.optim.AdamW(estim.parameters(), lr=1e-3,weight_decay=0.2))

estim.set_scheduler(torch.optim.lr_scheduler.StepLR(estim.optimizer, step_size=10, gamma=0.75))
estimators = [estim]

pipeline = Pipeline([
    ('preprocess', Preprocessor()),
    ('model', NBEATSModel(estimators))
])

model = pipeline.fit(x_train,y_train)
x_val['y'] = y_val
preds = model.predict(x_val)
score = wmae(preds['y'],preds['CustomNBEATS'],preds['IsHoliday'])
print(score)

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=10400` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


2308.7007630038447


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [18]:
! wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: azhgh22 (MLBeasts) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [20]:
import wandb
import joblib


fin_model = pipeline.fit(train.drop(columns=['Weekly_Sales'].copy()),train['Weekly_Sales'].copy())

joblib.dump(fin_model, "nbeats_run3.pkl")
wandb.init(project="Walmart Recruiting - Store Sales Forecasting", name="nbeats:run3")

wandb.config.update({
    'score_metric' : 'WMAE',
    'score_policy' : {
        'weight on holidays' : 5,
        'weight on non_holidays' : 1
    },
    'model' : 'nbeats',
    'learning_rate' : 0.001,
    'learning_scheduler' : 'True',
    'learning_scheduler_step' : 10,
    'learning_scheduler_gamma' : 0.75,
    'weight_decay' : 0,
    'batch_size' : 64,
    'max_depth' : 7,
    'max_steps' : 25 * 104,
    'input_size' : 15,
    'horizon': 48,
    'architecture' : ['identity', 'trend', 'seasonality'],
    'n_blocks' : [1,1,1],
    'random_state': 42,
    'objective' : 'reg:squarederror',
})

wandb.log({
    'val_wmae': score
})


artifact = wandb.Artifact(
    name="nbeats_run3",
    type="model",
)

artifact.add_file("nbeats_run3.pkl")
wandb.log_artifact(artifact)

wandb.finish()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2600` reached.
wandb: Currently logged in as: azhgh22 (MLBeasts) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


val_wmae,▁
val_wmae,2344.6255


In [21]:
pred = fin_model.predict(test)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [25]:
pred

,unique_id,ds,y,IsHoliday,CustomNBEATS
0,1_1,2012-11-02,0,0,31145.664062
1,1_1,2012-11-09,0,0,22807.623047
2,1_1,2012-11-16,0,0,20034.949219
3,1_1,2012-11-23,0,1,19596.896484
4,1_1,2012-11-30,0,0,19121.177734
...,...,...,...,...,...
115059,45_98,2013-06-28,0,0,824.601746
115060,45_98,2013-07-05,0,0,832.055359
115061,45_98,2013-07-12,0,0,831.456604
115062,45_98,2013-07-19,0,0,753.761353


In [22]:
test1 = test.copy()
test1['unique_id'] = test1["Store"].astype(str) + "_" + test1["Dept"].astype(str)
test1['ds'] = pd.to_datetime(test1['Date'])

In [23]:
merged = pd.merge(test1, pred, on=['unique_id','ds'],how='left')
merged['Id'] = test1["unique_id"].astype(str) + "_" + test1["ds"].astype(str)
merged['Weekly_Sales'] = merged['CustomNBEATS']

In [24]:
merged[['Id','Weekly_Sales']].to_csv('submission.csv', index=False)